<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js021_fasterMixMMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [2]:
#@title js021_fasterMixMMD2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.09.18 created,  last updated on 2024.04.28
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js021_fasterMixMMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js021_fasterMixMMD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.08 created, last updated on 2018.11.15
//    ver 0.0.1  2019.01.14 v1, last updated on 2021.05.10
//    ver 0.0.2  2021.10.30 v2, last updated on 2021.10.30
//    ver 0.0.3  2023.03.07 v3, last updated on 2023.08.20
//
// --------------------  molecular dynamics 2D
//
//  method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      (6) goto (1)
//
//  potential: Morse V(r) = D*((1-EXP(-A*(r-r0)))^2-1)
//                        = D*(EXP(-2*A*(r-r0))-2*EXP(-A*(r-r0)))
//     (D:dissociation energy, r0:bond length, A:width parameter A=SQR(k/(2*D))
//             force F(r) = -dV(r)/dr
//                        = 2*D*A*y*(y-1), y=EXP(-A*(r-r0))
//
//  for faster calculation: O(N) // fast calculation (without pre-registration): O(N^2)
//    ignore F(r) r>rCutoff
//    force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//    registration with pre-registration
//      pre-registration O(N), see preRegistration()
//        particles pre-regist into lattice section[i][j],
//        and register into reg[ni][k] every particle ni
//      registration reg[][] (see registration()), 'near' means r<rCutoff+20*2000*dt
//        reg[][] use 20 times, assuming particle max speed < 2000m/s
//    force calculation: sum up force(r) (r<rCutoff)
//
// --------------------
*/

const fasterMixMMD2D = (function(){ // ====================  fasterMixMMD2D Module  ====================

	const g_AMU = 1.66053904e-27;				// (kg) atomic mass unit
	const g_kB = 1.380649e-23;					// (J/K) Boltzmann's constant
	const g_EE = 1.602176634e-19;				// (C) electron charge, energy : 1(eV) = EE(J)
	const g_nMax = 8000;						// array max

	let g_molecKind1 = 3;						// 3:Fe
	let g_molecKind2 = 5;						// 5:Al
	let g_nParticles = 100;						// number of particles
	let g_sysTime = 0.0;						// (s) system time
	let g_timeStep =  5.0*1.0e-15;				// (s) time step
	let g_xMax = 6.0E-9;						// (m) x-Box size
	let g_yMax = 6.0E-9;						// (m) y-Box size
	let g_Nsx = 48;								// use pre-registration section(0 to Nsx,0 to Nsy)
	let g_Nsy = 48;								// use pre-registration section(0 to Nsx,0 to Nsy)
	let g_kineticEnergy = 0.0;					// (J) total kinetic energy
	let g_potentialEnergy = 0.0;				// (J) total potential energy
	let g_rCutoff = 1.0e-9;						// force cutoff length
	let g_hh = 1.0e-12;							// forceTable r-division

	const g_xx = dim1( g_nMax );				// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );				// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );				// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );				// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );				// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );				// (N) y-component of total force applied i-th particle
	const g_mas = dim1( g_nMax );				// (kg) mass of i-th particle
	const g_kind = dimInt1( g_nMax );			// kind of i-th particle
	const g_potentialTable = dim3( 21, 21, 1002 );	// potential table(ki,kj) [V[0], V[hh], V[2hh],..., V[rCutoff]]
	const g_forceTable = dim3( 21, 21, 1002 );	// force table(ki,kj) [F[0], F[hh], F[2hh],..., F[rCutoff]]
	const g_reg = dimInt2( g_nMax, 100 );		// reg[i][0]: total number of particles near i-th particle
												// reg[i][j]: particle number near i-th particle, (j>0)
	const g_section = dimInt3( 100, 100, 20 );	// section[i][j][0]: total number of particles in section[i][j]
												// section[i][j][k]: k-th particle number in section[i][j]

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}

	function dimInt3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Int32Array( nk );
			}
		}
		return a;
	}


	// --------------------  Morse potential data  --------------------

	//   0 mass(AU),      1 dd(eV),     2 aa(1/m), 3 r0(m),   4 col,     5 molecStr
	const g_Morse = [
		[ 183.85 *g_AMU, 0.9906*g_EE, 1.4116e10, 3.032e-10, "#6622ff", "W"  ], //  0
		[  95.94 *g_AMU, 0.8032*g_EE, 1.5079e10, 2.976e-10, "#4444ff", "Mo" ], //  1
		[  51.996*g_AMU, 0.4414*g_EE, 1.5721e10, 2.754e-10, "#0088ff", "Cr" ], //  2
		[  55.847*g_AMU, 0.4174*g_EE, 1.3885e10, 2.845e-10, "#00ffff", "Fe" ], //  3
		[  58.71 *g_AMU, 0.4205*g_EE, 1.4199e10, 2.780e-10, "#00ff88", "Ni" ], //  4
		[  26.98 *g_AMU, 0.2703*g_EE, 1.1646e10, 3.253e-10, "#00ff00", "Al" ], //  5
		[ 207.19 *g_AMU, 0.2348*g_EE, 1.1836e10, 3.733e-10, "#88ff00", "Pb" ], //  6
		[  63.54 *g_AMU, 0.3429*g_EE, 1.3588e10, 2.866e-10, "#cccc00", "Cu" ], //  7
		[ 107.87 *g_AMU, 0.3323*g_EE, 1.3690e10, 3.115e-10, "#bbbb66", "Ag" ], //  8
		[  40.08 *g_AMU, 0.1623*g_EE, 0.8054e10, 4.569e-10, "#ddaa00", "Ca" ], //  9
		[  87.62 *g_AMU, 0.1513*g_EE, 0.7878e10, 4.988e-10, "#ff8800", "Sr" ], // 10
		[ 137.34 *g_AMU, 0.1416*g_EE, 0.6570e10, 5.373e-10, "#ff4400", "Ba" ], // 11
		[  22.99 *g_AMU, 0.0633*g_EE, 0.5900e10, 5.336e-10, "#ff0000", "Na" ], // 12
		[  39.102*g_AMU, 0.0542*g_EE, 0.4977e10, 6.369e-10, "#ff0044", "K"  ], // 13
		[  85.47 *g_AMU, 0.0464*g_EE, 0.4298e10, 7.207e-10, "#ff0088", "Rb" ], // 14
		[ 132.905*g_AMU, 0.0449*g_EE, 0.4157e10, 7.557e-10, "#ff00cc", "Cs" ], // 15
		[  20.183*g_AMU, 0.0031*g_EE, 1.6500e10, 3.076e-10, "#ff00ff", "Ne" ], // 16
		[  39.948*g_AMU, 0.0104*g_EE, 1.3400e10, 3.816e-10, "#cc44ff", "Ar" ], // 17
		[  83.80 *g_AMU, 0.0141*g_EE, 1.2500e10, 4.097e-10, "#8844ff", "Kr" ], // 18
		[ 131.30 *g_AMU, 0.0200*g_EE, 1.2400e10, 4.467e-10, "#4488ff", "Xe" ], // 19
		[ 200.59 *g_AMU, 0.0734*g_EE, 1.4900e10, 3.255e-10, "#00ccff", "Hg" ] ]; // 20

	const g_rCollision = [];
	const g_rBond = [];
	const g_massOf = [];
	const g_strOf = [];
	const g_colorOf = [];		// g_colorOf[kind] : color of kind

	(function() {
		const n=g_Morse.length;

		for (let i=0; i<n; i++) {
			g_rCollision[i] = 0.5*g_Morse[i][3]/1.12246;
			g_rBond[i] = 0.5*g_Morse[i][3];
			g_massOf[i] = g_Morse[i][0];
			g_strOf[i] = g_Morse[i][5];
			g_colorOf[i] = g_Morse[i][4];
		}
	}());

	function colorStrOf(kind) {
		return "<span style='color:"+g_colorOf[kind]+"'>"+g_strOf[kind]+"</span>";
	}

	//--- set forceTable[kindi][kindj][ir] and potentialTable[kindi][kindj][ir]

	function setForceTable() {
		const hh=g_hh;
		for (let i=0; i<21; i++) {
			for (let j=0; j<21; j++) {
				const dd = Math.sqrt(g_Morse[i][1]*g_Morse[j][1]);
				const aa = 0.5*(g_Morse[i][2]+g_Morse[j][2]);
				const r0 = 0.5*(g_Morse[i][3]+g_Morse[j][3]);
				for (let ir=1; ir<=1001; ir++) {
					const r = ir*hh;
					const y = Math.exp(-aa*(r-r0));
					g_potentialTable[i][j][ir] = cutoff(r)*dd*y*(y-2.0); // V(r) = D*((1-y)^2-1) = D*y*(y-2)
					//g_forceTable[i][j][ir] = cutoff(r)*2.0*dd*aa*y*(y-1);
				}
				g_potentialTable[i][j][0] = g_potentialTable[i][j][1] + g_potentialTable[i][j][2];
				for (let ir=1; ir<=1000; ir++) {
					g_forceTable[i][j][ir] = -(g_potentialTable[i][j][ir+1] - g_potentialTable[i][j][ir-1])/(2.0*hh);
				}
				g_forceTable[i][j][1001] = -(0.0 - g_potentialTable[i][j][1000])/(2.0*hh);
				g_forceTable[i][j][0] = g_forceTable[i][j][1];
			}
		}
	}

	function cutoff(r) {
		let ret;
		if (r>0 && r<0.8*g_rCutoff) {
			ret = 1.0;
		} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
			ret = 0.5+0.5*Math.cos(Math.PI*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
		} else {
			ret = 0.0;
		}
		return ret;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme, boxSizeInNM, contTemp ) {
		setForceTable();
		g_sysTime = 0.0;
		g_xMax = boxSizeInNM*1.0e-9;
		g_yMax = boxSizeInNM*1.0e-9;

		let knd1,knd2;
		if (theme==0) { // Fe-Al
			knd1 = 3; knd2 = 5;
		} else if (theme==1) { // Ni-Pb
			knd1 = 4; knd2 = 6;
		} else if (theme==2) { // Ni-Cr
			knd1 = 4; knd2 = 2;
		} else if ( theme==3 ) { // W-Cu
			knd1 = 0; knd2 = 7;
		} else if ( theme==4 ) { // Al-Ca
			knd1 = 5; knd2 = 9;
		} else if ( theme==5 ) { // Fe-Fe
			knd1 = 3; knd2 = 3;
		} else { // else: Fe-Al
			knd1 = 3; knd2 = 5;
		}
		setCondition(knd1,knd2);
		ajustVelocity(contTemp);
	}

	function setCondition(knd1,knd2,contTemp) {
		g_molecKind1 = knd1;
		g_molecKind2 = knd2;
		g_rCutoff = Math.min(Math.max(6.0*g_rBond[g_molecKind1],6.0*g_rBond[g_molecKind2]),1.0e-9);
		const xtalSize = 0.4*g_xMax;
		const gap = 0.5e-9;
		const s = (g_xMax-2*xtalSize-gap)/2;
		const ss = s+xtalSize+gap;
		let ii = 0;
		ii = setCrystalBlock(ii, knd1, s, s, xtalSize, xtalSize, Math.PI/4);
		ii = setCrystalBlock(ii, knd2, s, ss, xtalSize, xtalSize, 0);
		ii = setCrystalBlock(ii, knd2, ss, s, xtalSize, xtalSize, Math.PI/8)
		g_nParticles = setCrystalBlock(ii, knd1, ss, ss, xtalSize, xtalSize, 3*Math.PI/8);
	}

	function setCrystalBlock(ii, knd, x0, y0, xLen, yLen, theta) {
		let iip = ii;
		const a = 0.98*2.0*g_rBond[knd];
		const b = 0.866025*a;
		let leng = xLen;
		if (leng<yLen) leng = yLen;
		leng = 1.5*leng;
		const nx = Math.floor(leng/b) + 1;
		const ny = Math.floor(leng/a) + 1;
		const sth = Math.sin(theta);
		const cth = Math.cos(theta);
		for (let i=0; i<nx; i++) {
			const x = b*i - leng/2.0;
			for (let j=0; j<ny; j++) {
				let y = a*j - leng/2.0;
				if ((i%2)==1) y = y + 0.5*a;
				const xp = x0 + xLen/2.0 + cth*x - sth*y;
				const yp = y0 + yLen/2.0 + sth*x + cth*y;
				if (xp>=x0 && xp<=x0+xLen && yp>=y0 && yp<=y0+yLen) {
					setParticle(iip, knd, xp, yp);
					iip = iip + 1;
				}
			}
		}
		return iip;
	}

	function setParticle(i, knd, x, y) {
		g_xx[i] = x;
		g_yy[i] = y;
		g_vx[i] = 200.0*normalRandom3();
		g_vy[i] = 200.0*normalRandom3();
		g_ffx[i] = 0.0;
		g_ffy[i] = 0.0;
		g_kind[i] = knd;
		g_mas[i] = g_massOf[knd];
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( tempMode, contTemp ) {

		if (tempMode==1) ajustVelocity(contTemp);
		//registerNearParticles(); // O(N^2)
		registration(); // O(N)
		for (let i=0; i<20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
	}

	function moveParticles(dt) {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i]
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*3.418e-10;
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0; g_ffy[i]=0;
		}

		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j];
				const rij = Math.sqrt(xij*xij+yij*yij);
				if (rij<g_rCutoff) {
					const f = force(rij,g_kind[i],g_kind[j]);
					const fxij = f*xij/rij;
					const fyij = f*yij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
				}
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function force(r,kndi,kndj) { // forceTable - linear interporation
		const hh=g_hh;
		const ir = Math.floor(r/hh);
		const a = r - ir*hh;
		g_potentialEnergy += ((hh-a)*g_potentialTable[kndi][kndj][ir] + a*g_potentialTable[kndi][kndj][ir+1])/hh;
		return ((hh-a)*g_forceTable[kndi][kndj][ir] + a*g_forceTable[kndi][kndj][ir+1])/hh;
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0); // epsilonOfAr = 1.711e-21
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	//--- registeration: O(N^2)

	function registerNearParticles() { // no use
		const nn=g_nParticles;
		const rCut = g_rCutoff+20*2000*g_timeStep;
		const rcut2 = rCut*rCut;
		for (let i=0; i<nn-1; i++) {
			let k = 1;
			for (let j=i+1; j<nn; j++) {
				const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]);
				if (r2<rcut2) {
					g_reg[i][k] = j;
					k = k + 1;
				}
			}
			g_reg[i][0] = k;
		}
	}

	function maxNearParticles() {
		let mx=0;
		for (let i=1; i<nParticles-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}

	//--- registration with preRegistration: O(N)

	function registration() {
		const nn=g_nParticles;

		preRegistration();
		const rreg = g_rCutoff+20*2000*g_timeStep;
		const rreg2 = rreg*rreg;
		for (let ipp=0; ipp<nn-1; ipp++) {
			let kp = 1;
			let i0 = Math.floor(g_Nsx*(g_xx[ipp]-rreg)/g_xMax);
			if (i0<0) i0 = 0;
			let i1 = Math.floor(g_Nsx*(g_xx[ipp]+rreg)/g_xMax );
			if (i1>=g_Nsx) i1 = g_Nsx-1;
			let j0 = Math.floor(g_Nsy*(g_yy[ipp]-rreg)/g_yMax );
			if (j0<0) j0 = 0;
			let j1 = Math.floor(g_Nsy*(g_yy[ipp]+rreg)/g_yMax );
			if (j1>=g_Nsy) j1 = g_Nsy-1;
			for (let i=i0; i<=i1; i++) {
				for (let j=j0; j<=j1; j++) {
					for (let iq=1; iq<=g_section[i][j][0]; iq++) {
						const jp = g_section[i][j][iq];
						if (jp>ipp) {
							const r2=(g_xx[ipp]-g_xx[jp])*(g_xx[ipp]-g_xx[jp])+(g_yy[ipp]-g_yy[jp])*(g_yy[ipp]-g_yy[jp]);
							if (r2<rreg2) {
								g_reg[ipp][kp] = jp;
								kp = kp + 1;
							}
						}
					}
				}
			}
			g_reg[ipp][0] = kp;
		}
	}

	function preRegistration() {
		const nn=g_nParticles;

		for (let i=0; i<g_Nsx; i++) {
			for (let j=0; j<g_Nsy; j++) {
				g_section[i][j][0] = 0;
			}
		}
		for (let ipp=0; ipp<nn; ipp++) {
			let i = Math.floor(g_Nsx*g_xx[ipp]/g_xMax);
			if (i>=g_Nsx) i = g_Nsx-1;
			let j = Math.floor(g_Nsy*g_yy[ipp]/g_yMax);
			if (j>=g_Nsy) j = g_Nsy-1;
			const iq = g_section[i][j][0] + 1;
			g_section[i][j][0] = iq;
			g_section[i][j][iq] = ipp
		}
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn=g_nParticles;
		let ek= 0.0;  //kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB);
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme, boxSizeInNM, contTemp )
		evolve:			timeEvolution,			// timeEvolution( tempMode, contTemp )

		setTemp:		ajustVelocity,			// ajustVelocity( temp )

		getSysParam:	function() { return [ g_molecKind1, g_molecKind2, g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getKindStr:		function(kind) { return g_strOf[kind]; },

		getBondLength:	function(i,j) { return ( g_rBond[g_kind[i]] + g_rBond[g_kind[j]] ); },
		getNearList:	function(i) { return g_reg[i]; },
		getAtomData:	function(i) { return [ g_kind[i], g_rCollision[g_kind[i]], g_rBond[g_kind[i]] ]; },

		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
	};

})(); // ====================  fasterMixMMD2D end  ====================


const js021 = (function(){ // ====================  js Module  ====================

	const theModule = fasterMixMMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 0;			// 0:Fe-Al, 1:Ni-Pb, 2:Ni-Cr, 3:W-Cu, 4:Al-Ca, 5:Fe
	let v_BoxSizeInNM = 8.0;
	let v_contTemp = 300.0;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control

	let p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax; // = theModule.getSysParam();
	let sysTime, temperature, kineticEnergy, potentialEnergy;
	let nowData = [];
  let kindList = [];
  let xxList = [];
  let yyList = [];
  let vxList = [];
  let vyList = [];

	let nCalc = 1;
	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

	let breakFlag = false;
	let perticleFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_BoxSizeInNM, v_contTemp );
			[ p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
			perticleFlag = true;
		}

		if ( !pauseFlag ) {
			for(let i=0; i<nCalc; i++ ) {
				theModule.evolve( v_tempMode, v_contTemp );
			}
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_tempMode, v_contTemp );
		}

		draw( ctx, dispMode );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

	function setParticlesData() {
		nowData = [ sysTime, temperature, kineticEnergy, potentialEnergy ];
		kindList = [];
		xxList = [];
		yyList = [];
		vxList = [];
		vyList = [];
    for (let i=0; i<p_nParticles; i++) {
			let x, y, vx, vy;
      kindList[i] = theModule.getAtomData(i)[0];
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
		}
	}


	// --------------------  draw  --------------------

	const ballColor = [
		/*0:W */ "#6622ff", /*1:Mo*/ "#4444ff", /*2:Cr*/ "#0088ff", /*3:Fe*/ "#00ffff", /*4:Ni*/ "#00ff88",
		/*5:Al*/ "#00ff00", /*6:Pb*/ "#88ff00", /*7:Cu*/ "#cccc00", /*8:Ag*/ "#bbbb66", /*9:Ca*/ "#ddaa00" ];

	function draw( ctx, dispMode ) {
		const xp = 40, yp = 5, xSize = 400, ySize = 400, scale = xSize/p_xMax, yTextPos= 430;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		if ( dispMode==0 ) {
			drawBalls( ctx, scale, xp, yp );
		} else if ( dispMode==1 || dispMode==2 ) {
			strokeBalls( ctx, scale, xp, yp );
			drawBonds( ctx, dispMode-1, scale, xp, yp );
		}

		// caption
		[ sysTime, temperature, kineticEnergy, potentialEnergy ] = theModule.getNow();

		ctx.fillStyle = ballColor[p_kind1];
		ctx.fillText(theModule.getKindStr(p_kind1), xp, yTextPos+20);
		if ( p_kind1 != p_kind2 ) {
			ctx.fillStyle = ballColor[p_kind2];
			ctx.fillText(theModule.getKindStr(p_kind2), xp+30, yTextPos+20);
		}
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos);
		ctx.fillText(`Temp = ${temperature.toFixed(1)} (K)`, xp+200, yTextPos);
		ctx.fillText(`,   N = ${p_nParticles}`, xp+60, yTextPos+20);
		ctx.fillText(`cont. Temp = ${v_contTemp.toFixed(0)} (K)`, xp+200, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(1)} x ${(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, xp+200, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"
	}

	function drawBalls( ctx, scale, xp, yp ) {
		const nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		for (let i=0; i<nn; i++) {
			let x, y; [ x, y ] = theModule.getPosition(i);
			let kind, rCollision, rBond; [ kind, rCollision, rBond ] = theModule.getAtomData(i);
			ctx.fillStyle = ballColor[kind];
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, rCollision*scale, 0, twoPi, false);
			ctx.fill();
		}
	}

	function strokeBalls( ctx, scale, xp, yp ) {
		const nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		const d0 = theModule.getBondLength();
		for (let i=0; i<nn; i++) {
			let x, y; [ x, y ] = theModule.getPosition(i);
			let kind, rCollision, rBond; [ kind, rCollision, rBond ] = theModule.getAtomData(i);
			ctx.strokeStyle = "#444444";
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, rBond*scale, 0, twoPi, false);
			ctx.stroke();
		}
	}

	function drawBonds( ctx, bondMode, scale, xp, yp ) { // single kind only - using nearList
		const nn = p_nParticles, yMax = p_yMax, pi = Math.PI;
		for (let i=0; i<nn-1; i++) {
			let xi, yi; [ xi, yi ] = theModule.getPosition(i);
			const nearList = theModule.getNearList(i);
			for (let k=1; k<nearList[0]; k++) {
				const j = nearList[k];
				let xj, yj; [ xj, yj ] = theModule.getPosition(j);
				const xij = xi-xj, yij = yi-yj;
				const dij = Math.sqrt(xij*xij+yij*yij);
				const d0 = theModule.getBondLength(i,j);
				if ( dij<1.2*d0 ) {
					let hue;
					if ( bondMode==0 ) { // bond color -- length
						hue = Math.floor((dij/d0-1.0)*900+120);
					} else if ( bondMode==1 ) { // bond color -- direction
						const th = 3.0*(Math.atan2(yij,xij)+0.5*pi)/pi;
						hue = Math.floor((th-Math.floor(th))*360.0);
					}
					if (hue<0) hue = 0;
					if (hue>270) hue = 270;
					const color = `hsl(${hue},100%,50%)`;
					drawLine(ctx, xi*scale+xp, (yMax-yi)*scale+yp, xj*scale+xp, (yMax-yj)*scale+yp, color );
				}
			}
		}
	}

	function drawLine(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setBoxSize() {  // select box size
		const slct = 0 + document.getElementById("slct_boxSize").selectedIndex;
		v_BoxSizeInNM = 6.0 + 2.0*slct;
		resetFlag = true;
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, temperature, kineticEnergy, potentialEnergy ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ nowData, kindList, xxList, yyList, vxList, vyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setBoxSize:		setBoxSize,		// setBoxSize()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()

	  breakLoop: breakLoop, // breakLoop();
	  pysetTheme: pysetTheme, // pysetTheme( theme )
	  pysetTempMode: pysetTempMode, // pysetTempMode( mode )
	  pysetTemperature: pysetTemperature, // pysetTemperature( temp )
	  pysetDispMode: pysetDispMode, // pysetDispMode( mode )
	  pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature, kineticEnergy, potentialEnergy ]
		pygetParticlesList, pygetParticlesList, //() :return [ nowData, kindList, xxList, yyList, vxList, vyList ]
	};

})(); // ====================  js021 module end  ====================


const js = js021;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js021] faster O(N)-calc. molecular dynamics 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme: </label>
<select id="slct_theme" onChange="js.setTheme()">
<option>Fe-Al</option><option>Ni-Pb</option><option>Ni-Cr</option><option>W-Cu</option>
<option>Al-Ca</option><option>Fe</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>box =</label>
<select id="slct_boxSize" onChange="js.setBoxSize()">
<option>6.0x6.0</option><option selected>8.0x8.0</option><option>10.0x10.0</option>
<option>12.0x12.0</option><option>14.0x14.0</option><option>16.0x16.0</option>
<option>18.0x18.0</option><option>20.0x20.0</option>
</select>
(nm)
    <span style="margin-right: 20px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>Temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont.Temp:</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="600" value="300" step="2"
style="width:360px" oninput="js.setContTemp()">
<br>

<label> disp. mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>ball</option><option>bond length</option><option>bond direction</option>
</select>
    <span style="margin-right: 80px;"></span>
<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# simulator run
exec_html_js()
print("-- start --")

# get temporally data and print
for i in range(10):
  [ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = ke + pe
  print(f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  K = {ke:9.6e},  U = {pe:9.6e},  energy = {energy:9.6e} (J)')
  time.sleep(2)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
#  change theme and  dispMode

import time

themeList = [ '0: Fe-Al', '1: Ni-Pb', '2: Ni-Cr' ]
dispModeList = [ 'ball', 'bond length', 'bond direction' ]

# simulator run
exec_html_js()
print("-- start --")

# change theme
for theme in [ 0, 1, 2 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print("-- theme: ", themeList[theme], " --" )
  # change dispMode
  for dispMode in [ 0, 1, 2 ]:
    eval_js('js.pysetDispMode({})'.format(dispMode) )
    print("  -- dispMode:", dispModeList[dispMode], " --")
    #get data and print
    [ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData({})'.format(i) )
    energy = ke + pe
    print(f'\t time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  K = {ke:9.6e},  U = {pe:9.6e},  energy = {energy:9.6e} (J)')
    time.sleep(2)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# get particles data and save

import time
import numpy as np

# simulator run
exec_html_js()
print("-- start --")

# wait 10 sec
time.sleep(10)

# get particle data
print("-- get particles data --")
[ nowData, kindList, xxList, yyList, vxList, vyList ] = eval_js('js.pygetParticlesList()')
[ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData({})'.format(i) )
energy = ke + pe
print(f'got time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  K = {ke:9.6e},  U = {pe:9.6e},  energy = {energy:9.6e} (J)')
print(f'len(kindList) ={len(kindList):>4d}, len(xxList) ={len(xxList):>4d}, len(yyList) ={len(yyList):>4d}, len(vxList) ={len(vxList):>4d},  len(vyList) ={len(vyList):>4d}')

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

# convert [ kindList, xxList, yyList, vxList, vyList ] to np_data
np_data = np.array([ kindList, xxList, yyList, vxList, vyList ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js021_data.npy'" )
np.save( 'js021_data.npy', np_data )

In [ ]:
# load particles data and set numpy array

import numpy as np

# atom kind data
atomList = [ '0:W', '1:Mo', '2:Cr', '3:Fe', '4:Ni', '5:Al', '6:Pb', '7:Cu', '8:Ag', '9:Ca' ]
colorList = np.array([
    [ 0x00, 0x00, 0xff ], [ 0x00, 0x40, 0xff ], [ 0x00, 0x80, 0xff ], [ 0x00, 0xc0, 0xff ], [ 0x00, 0xff, 0xff ],           # W,  Mo, Cr, Fe, Ni
    [ 0x00, 0xff, 0xc0 ], [ 0x00, 0xff, 0x80 ], [ 0x00, 0xff, 0x40 ], [ 0x00, 0xff, 0x00 ], [ 0x40, 0xff, 0x00 ], ]) / 255  # Al, Pb, Cu, Ag, Ca
sizeList = np.array([ 3.032, 2.976, 2.754, 2.845, 2.780, 3.253, 3.733, 2.866, 3.115, 4.569 ])

# box size
x_box_size, y_box_size = 8.0, 8.0  # (nm)

# load np_data <-- np.array([ kindList, xxList, yyList, vxList, vyList ])
print("-- load data --")
loaded_data = np.load('js021_data.npy')

# set numpy array
Kind = loaded_data[0].astype(int)  # kind of every atom
X = loaded_data[1] * 1e9           # (nm) x-position of every atom
Y = loaded_data[2] * 1e9           # (nm) y-position of every atom
U = loaded_data[3]                 # (m/s) x-velocity of every atom
V = loaded_data[4]                 # (m/s) y-velocity of every atom
Vabs = np.sqrt(U**2 + V**2)        # (m/s) speed of every atom
C = colorList[Kind]                # color of every atom
S = sizeList[Kind]                 # size of every atom
print(f'shape of Kind:{Kind.shape},  X:{X.shape},  Y:{Y.shape},  U:{U.shape},  V:{V.shape}')

In [ ]:
# plot particle position

import numpy as np
import matplotlib.pyplot as plt


# scatter plot
fig = plt.figure(figsize=(8, 8))
plt.scatter(X, Y, c=C, s=S*50)
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)
plt.xlabel('x ( nm )')
plt.ylabel('y ( nm )')
plt.title('Fe-Al crystals')
plt.show()

In [ ]:
# velocity space

import numpy as np
import matplotlib.pyplot as plt

# scatter plot
vmax = 1500
fig = plt.figure(figsize=(8, 8))
plt.scatter(U, V, c=C )
plt.xlabel('vx ( m/s )')
plt.ylabel('vy ( m/s )')
plt.title('velocity space')
plt.xlim(-vmax, vmax)
plt.ylim(-vmax, vmax)
plt.show()

In [ ]:
# histogram of Vabs
# colab AI wrote:
# prompt: hist Vabs Fe(kind==3), Al(kind==5)  in one figure (c=C)

# hist Vabs Fe(kind==3), Al(kind==5) in one figure (c=C)
# Fe: red, Al: blue

# get Fe(kind==3), Al(kind==5) data
Fe_mask = Kind == 3
Al_mask = Kind == 5
Fe_Vabs = Vabs[Fe_mask]
Al_Vabs = Vabs[Al_mask]

# plot
fig = plt.figure(figsize=(6, 6))
plt.hist(Fe_Vabs, bins=50, color='red', alpha=0.5, label='Fe')
plt.hist(Al_Vabs, bins=50, color='blue', alpha=0.5, label='Al')
plt.xlabel('speed (m/s)')
plt.ylabel('counts')
plt.legend()
plt.show()

In [ ]:
# Fe-Fe, Al-Al, Fe-Al bond length distribution
# colab AI wrote:
# prompt: bond length d_Fe_Fe < sizeList[3]*0.12,.. d_Fe_Al <(sizeList[3] + sizeList[5])/2 * 0.12, then get bond length distribution of Fe-Fe, Al-Al and Fe-Al

# get Fe-Fe bond length
Fe_mask = Kind == 3
Fe_X = X[Fe_mask]
Fe_Y = Y[Fe_mask]
Fe_bond_length = np.sqrt((Fe_X[1:] - Fe_X[:-1])**2 + (Fe_Y[1:] - Fe_Y[:-1])**2)
Fe_bond_length = Fe_bond_length[Fe_bond_length < sizeList[3] * 0.12]

# get Al-Al bond length
Al_mask = Kind == 5
Al_X = X[Al_mask]
Al_Y = Y[Al_mask]
Al_bond_length = np.sqrt((Al_X[1:] - Al_X[:-1])**2 + (Al_Y[1:] - Al_Y[:-1])**2)
Al_bond_length = Al_bond_length[Al_bond_length < (sizeList[3] + sizeList[5]) / 2 * 0.12]

# get Fe-Al bond length
Fe_Al_mask = (Kind == 3) + (Kind == 5)
Fe_Al_X = X[Fe_Al_mask]
Fe_Al_Y = Y[Fe_Al_mask]
Fe_Al_bond_length = np.sqrt((Fe_Al_X[1:] - Fe_Al_X[:-1])**2 + (Fe_Al_Y[1:] - Fe_Al_Y[:-1])**2)
Fe_Al_bond_length = Fe_Al_bond_length[Fe_Al_bond_length < (sizeList[3] + sizeList[5]) / 2 * 0.12]

# plot bond length distribution
fig = plt.figure(figsize=(6, 6))
plt.hist(Fe_bond_length, bins=50, color='red', alpha=0.5, label='Fe-Fe')
plt.hist(Al_bond_length, bins=50, color='blue', alpha=0.5, label='Al-Al')
plt.hist(Fe_Al_bond_length, bins=50, color='green', alpha=0.5, label='Fe-Al')
plt.xlabel('bond length (nm)')
plt.ylabel('counts')
plt.legend()
plt.show()

In [ ]:
# prompt: in '# Fe-Fe, Al-Al, Fe-Al bond length distribution', add number of bond in every bond kind

# get Fe-Fe bond length
Fe_mask = Kind == 3
Fe_X = X[Fe_mask]
Fe_Y = Y[Fe_mask]
Fe_bond_length = np.sqrt((Fe_X[1:] - Fe_X[:-1])**2 + (Fe_Y[1:] - Fe_Y[:-1])**2)
Fe_bond_length = Fe_bond_length[Fe_bond_length < sizeList[3] * 0.12]

# get Al-Al bond length
Al_mask = Kind == 5
Al_X = X[Al_mask]
Al_Y = Y[Al_mask]
Al_bond_length = np.sqrt((Al_X[1:] - Al_X[:-1])**2 + (Al_Y[1:] - Al_Y[:-1])**2)
Al_bond_length = Al_bond_length[Al_bond_length < (sizeList[3] + sizeList[5]) / 2 * 0.12]

# get Fe-Al bond length
Fe_Al_mask = (Kind == 3) + (Kind == 5)
Fe_Al_X = X[Fe_Al_mask]
Fe_Al_Y = Y[Fe_Al_mask]
Fe_Al_bond_length = np.sqrt((Fe_Al_X[1:] - Fe_Al_X[:-1])**2 + (Fe_Al_Y[1:] - Fe_Al_Y[:-1])**2)
Fe_Al_bond_length = Fe_Al_bond_length[Fe_Al_bond_length < (sizeList[3] + sizeList[5]) / 2 * 0.12]

# count bond number
num_Fe_Fe = len(Fe_bond_length)
num_Al_Al = len(Al_bond_length)
num_Fe_Al = len(Fe_Al_bond_length)

# plot bond length distribution
fig = plt.figure(figsize=(6, 6))
plt.hist(Fe_bond_length, bins=50, color='red', alpha=0.5, label='Fe-Fe (' + str(num_Fe_Fe) + ')')
plt.hist(Al_bond_length, bins=50, color='blue', alpha=0.5, label='Al-Al (' + str(num_Al_Al) + ')')
plt.hist(Fe_Al_bond_length, bins=50, color='green', alpha=0.5, label='Fe-Al (' + str(num_Fe_Al) + ')')
plt.xlabel('bond length (nm)')
plt.ylabel('counts')
plt.legend()
plt.show()

In [ ]:
# data analysis with pandas
# colab AI wrote:
# prompt: corr, scatter matrix X, Y, U, V and print 1/sqrt(N) with pandas

import pandas as pd
import numpy as np

# calculate correlation matrix
corr = np.corrcoef(np.array([X, Y, U, V]))
print(corr)

# print 1/sqrt(N)
print("1/sqrt(N) = ",1 / np.sqrt(len(X)))

# create a scatter matrix
pd.plotting.scatter_matrix(pd.DataFrame({'X': X, 'Y': Y, 'U': U, 'V': V}))


In [ ]:
# plot paeticle position ( atom color: number of bond )
# colab AI wrote:
# prompt: plot particle position with plt.scatter(X, Y, c=number of bond, cmap= 'jet', s=S*50)

# get number of bond for each atom
bond_num = np.zeros(len(X))
for i in range(len(X)):
  # Fe-Fe bond
  if Kind[i] == 3:
    for j in range(i+1, len(X)):
      if Kind[j] == 3:
        if np.sqrt((X[i] - X[j])**2 + (Y[i] - Y[j])**2) < sizeList[3] * 0.12:
          bond_num[i] += 1
          bond_num[j] += 1
  # Al-Al bond
  if Kind[i] == 5:
    for j in range(i+1, len(X)):
      if Kind[j] == 5:
        if np.sqrt((X[i] - X[j])**2 + (Y[i] - Y[j])**2) < (sizeList[3] + sizeList[5]) / 2 * 0.12:
          bond_num[i] += 1
          bond_num[j] += 1
  # Fe-Al bond
  if Kind[i] == 3 or Kind[i] == 5:
    for j in range(len(X)):
      if Kind[j] == 3 or Kind[j] == 5:
        if i != j:
          if np.sqrt((X[i] - X[j])**2 + (Y[i] - Y[j])**2) < (sizeList[3] + sizeList[5]) / 2 * 0.12:
            bond_num[i] += 1

# plot particle position
fig = plt.figure(figsize=(8, 8))
plt.scatter(X, Y, c=bond_num, cmap='jet', s=S*50)
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)
plt.xlabel('x ( nm )')
plt.ylabel('y ( nm )')
plt.title('Fe-Al crystals - atom color: number of bond')
plt.show()
